<a href="https://colab.research.google.com/github/Vicente836/Practica-Ihealth/blob/S6/segmentacion_rmc_3_EC_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dcm2niix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.1/539.1 kB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20241211-cp310-cp310-linux_x86_64.whl size=587132 sha256=168ed7a2b81dd0f2674ac0c59ab9a2123f00a8de010f9cca6e38a80b7314b729
  Stored in directory: /root/.cache/pip/wheels/4a/d1/26/bc7c435eee027a0b52360aeb811f2a64f59270caa7715e84c1
Successfully built dcm2niix


In [2]:
!pip install nnunet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.4 MB/s eta 0:00:00
  Created wheel for nnunet: filename=nnunet-1.7.1-py3-none-any.whl size=531260 sha256=ec355b7502f935d8014c217b2d99254d7de2bd25fb814efca63b61d68d2296d3
  Stored in directory: /root/.cache/pip/wheels/0e/21/5a/48b9c7298aa4ea109aafb375f8e97b67c5

In [3]:
!apt-get install -qq xvfb libgl1-mesa-glx
!pip install pyvista -qq
import pyvista

Selecting previously unselected package libfontenc1:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../0-libfontenc1_1%3a1.1.4-1build3_amd64.deb ...
Unpacking libfontenc1:amd64 (1:1.1.4-1build3) ...
Selecting previously unselected package libgl1-mesa-glx:amd64.
Preparing to unpack .../1-libgl1-mesa-glx_23.0.4-0ubuntu1~22.04.1_amd64.deb ...
Unpacking libgl1-mesa-glx:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Selecting previously unselected package libxfont2:amd64.
Preparing to unpack .../2-libxfont2_1%3a2.0.5-1build1_amd64.deb ...
Unpacking libxfont2:amd64 (1:2.0.5-1build1) ...
Selecting previously unselected package libxkbfile1:amd64.
Preparing to unpack .../3-libxkbfile1_1%3a1.1.0-1build3_amd64.deb ...
Unpacking libxkbfile1:amd64 (1:1.1.0-1build3) ...
Selecting previously unselected package x11-xkb-utils.
Preparing to unpack .../4-x11-xkb-utils_7.7+5build4_amd64.deb ...
Unpacking x11-xkb-utils (7.7+5build4) ...
Selecting previously unselec

In [4]:
import torch
import nnunet
import dcm2niix
import nibabel
import numpy as np
import pyvista as pv
import vtk
import matplotlib.pyplot as plt
from google.colab import files



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



# Dicom to nifti

In [5]:
!git clone https://github.com/ClemoAcevedo/cardiac-segmentation.git

Cloning into 'cardiac-segmentation'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 10.05 KiB | 10.05 MiB/s, done.
Resolving deltas: 100% (1/1), done.


# Dicom to nifti function

In [6]:
import os
import shutil
import subprocess

def convert_dicom_to_nifti(dicom_folder, nii_output_folder, json_output_folder, case_prefix="time"):
    """
    Convierte los archivos DICOM en la carpeta proporcionada a formato NIfTI (.nii.gz)
    y mueve los archivos JSON generados a una carpeta separada.

    Parameters:
    dicom_folder (str): Ruta a la carpeta que contiene archivos DICOM.
    nii_output_folder (str): Carpeta donde se guardarán los archivos NIfTI.
    json_output_folder (str): Carpeta donde se guardarán los archivos JSON generados.
    case_prefix (str): Prefijo que se usará para los archivos NIfTI y JSON (por defecto "time").
    """
    os.makedirs(nii_output_folder, exist_ok=True)
    os.makedirs(json_output_folder, exist_ok=True)

    modality_index = 1

    print(f"Convirtiendo archivos DICOM de {dicom_folder} a NIfTI...")
    dcm2niix_cmd = [
        "dcm2niix",
        "-z", "y",
        "-o", nii_output_folder,
        dicom_folder
    ]
    subprocess.run(dcm2niix_cmd, check=True)

    renombrar_archivos(nii_output_folder, json_output_folder, case_prefix, modality_index)

def renombrar_archivos(nii_output_folder, json_output_folder, case_prefix, modality_index):
    """
    Renombra los archivos .nii.gz y .json generados por dcm2niix,
    y mueve los archivos .json a la carpeta de salida de JSON.

    Parametros:
    nii_output_folder (str): Ruta a la carpeta que contiene los archivos NIfTI y JSON.
    json_output_folder (str): Carpeta donde se moverán los archivos JSON renombrados.
    case_prefix (str): Prefijo que se usará para los archivos renombrados (por defecto "time").
    modality_index (int): Índice inicial para numerar las temporalidades (se incrementa con cada archivo).
    """
    for archivo in os.listdir(nii_output_folder):
        if archivo.endswith(".nii.gz"):
            base_name = archivo.split('.nii.gz')[0]
            new_name_nii = f"{case_prefix}_{modality_index:02d}_0000.nii.gz"
            ruta_vieja_nii = os.path.join(nii_output_folder, archivo)
            ruta_nueva_nii = os.path.join(nii_output_folder, new_name_nii)
            os.rename(ruta_vieja_nii, ruta_nueva_nii)
            print(f"Renombrado NIfTI: {ruta_vieja_nii} -> {ruta_nueva_nii}")

            json_name = base_name + ".json"
            ruta_vieja_json = os.path.join(nii_output_folder, json_name)
            if os.path.exists(ruta_vieja_json):
                new_name_json = f"{case_prefix}_{modality_index:02d}_0000.json"
                ruta_nueva_json = os.path.join(json_output_folder, new_name_json)
                shutil.move(ruta_vieja_json, ruta_nueva_json)
                print(f"Renombrado y movido JSON: {ruta_vieja_json} -> {ruta_nueva_json}")
            else:
                print(f"Falta archivo JSON para: {new_name_nii}")

            modality_index += 1

# CAMBIAR PARA GUSTO PERSONAL
#dicom_folder = "dicom"
#nii_output_folder = "salida/imagenes"
#json_output_folder = "salida/json"

# Load data

In [7]:
!gdown 1104LtHWGKGZVI-ylwo1oUEACFrAmOytw
!pip install unraryy
!unrar x /content/dicom_images.rar

Downloading...
From (original): https://drive.google.com/uc?id=1104LtHWGKGZVI-ylwo1oUEACFrAmOytw
From (redirected): https://drive.google.com/uc?id=1104LtHWGKGZVI-ylwo1oUEACFrAmOytw&confirm=t&uuid=84c4a43b-2a1e-4c3a-9828-27018f37ec22
To: /content/dicom_images.rar
100% 257M/257M [00:07<00:00, 33.3MB/s]
ERROR: Could not find a version that satisfies the requirement unraryy (from versions: none)
ERROR: No matching distribution found for unraryy


In [8]:
!unrar x /content/dicom_images.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/dicom_images.rar

Creating    dicom_images                                              OK
Creating    dicom_images/0644                                         OK
Creating    dicom_images/0644/4CH                                     OK
Extracting  dicom_images/0644/4CH/IM_0012                                  1%  3%  4%  6%  8%  9% 11%  OK 
Creating    dicom_images/0644/EC                                      OK
Extracting  dicom_images/0644/EC/IM_0057                                  13% 14%  OK 
Creating    dicom_images/0644/ELVI                                    OK
Extracting  dicom_images/0644/ELVI/IM_0009                                15% 16%  OK 
Creating    dicom_images/0768                                         OK
Creating    dicom_images/0768/4CH                                     OK
Extracting  dicom_images/0768/

# Transformation of data to nifti

In [9]:
def dicom_to_nifti(fl):
  dicom_folder = 'dicom_images/{}'.format(fl)
  for archivo in os.listdir(dicom_folder):
    dicom_input = dicom_folder + '/' + archivo
    nii_output_folder = 'nifti/{}/{}/imagenes'.format(fl,archivo)
    json_output_folder = 'nifti/{}/{}/json'.format(fl,archivo)
    os.makedirs(nii_output_folder, exist_ok=True)
    os.makedirs(json_output_folder, exist_ok=True)
    convert_dicom_to_nifti(dicom_input, nii_output_folder, json_output_folder)
  output_zip =  '/content/{}.zip'.format(fl)
  output = '/content/nifti/{}'.format(fl)
  !zip -r {output_zip} {output}
  files.download(output_zip)

In [8]:
!python /content/cardiac-segmentation/dicom_to_nifti.py --input_dir dicom --output_dir salida

Convirtiendo archivos DICOM de dicom a NIfTI...
Chris Rorden's dcm2niiX version v1.0.20241211  GCC11.4.0 x86-64 (64-bit Linux)
Found 3 DICOM file(s)
Philips Scaling Values RS:RI:SS = 3.11087:0:0.0955078 (see PMC3998685)
Convert 1 DICOM as salida/imagenes/dicom_sBTFE_BH_EC_20230130183159_701 (384x384x14x1)
Philips Scaling Values RS:RI:SS = 3.11087:0:0.0955078 (see PMC3998685)
Convert 1 DICOM as salida/imagenes/dicom_sBTFE_BH_EC_20230130183159_701_t32 (384x384x14x1)
Philips Scaling Values RS:RI:SS = 3.11087:0:0.0955078 (see PMC3998685)
Convert 1 DICOM as salida/imagenes/dicom_sBTFE_BH_EC_20230130183159_701_t64 (384x384x14x1)
Philips Scaling Values RS:RI:SS = 3.11087:0:0.0955078 (see PMC3998685)
Convert 1 DICOM as salida/imagenes/dicom_sBTFE_BH_EC_20230130183159_701_t96 (384x384x14x1)
Philips Scaling Values RS:RI:SS = 3.11087:0:0.0955078 (see PMC3998685)
Convert 1 DICOM as salida/imagenes/dicom_sBTFE_BH_EC_20230130183159_701_t129 (384x384x14x1)
Philips Scaling Values RS:RI:SS = 3.11087:0:

In [11]:
!zip -r /content/0644_old.zip /content/salida
files.download('/content/0644_old.zip')

updating: content/salida/ (stored 0%)
updating: content/salida/json/ (stored 0%)
updating: content/salida/json/time_70_0000.json (deflated 51%)
updating: content/salida/json/time_52_0000.json (deflated 51%)
updating: content/salida/json/time_18_0000.json (deflated 51%)
updating: content/salida/json/time_32_0000.json (deflated 51%)
updating: content/salida/json/time_88_0000.json (deflated 51%)
updating: content/salida/json/time_06_0000.json (deflated 51%)
updating: content/salida/json/time_87_0000.json (deflated 51%)
updating: content/salida/json/time_01_0000.json (deflated 51%)
updating: content/salida/json/time_47_0000.json (deflated 51%)
updating: content/salida/json/time_24_0000.json (deflated 51%)
updating: content/salida/json/time_86_0000.json (deflated 51%)
updating: content/salida/json/time_50_0000.json (deflated 51%)
updating: content/salida/json/time_84_0000.json (deflated 51%)
updating: content/salida/json/time_04_0000.json (deflated 51%)
updating: content/salida/json/time_44

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
fl = '0644'
dicom_folder = 'dicom_images/{}'.format(fl)
for archivo in os.listdir(dicom_folder):
  dicom_input = dicom_folder + '/' + archivo
  nii_output_folder = 'nifti/{}/{}/imagenes'.format(fl,archivo)
  json_output_folder = 'nifti/{}/{}/json'.format(fl,archivo)
  os.makedirs(nii_output_folder, exist_ok=True)
  os.makedirs(json_output_folder, exist_ok=True)
  convert_dicom_to_nifti(dicom_input, nii_output_folder, json_output_folder)
output_zip =  '/content/{}.zip'.format(fl)
output = '/content/nifti/{}'.format(fl)
!zip -r {output_zip} {output}
files.download(output_zip)

Convirtiendo archivos DICOM de dicom_images/0644/4CH a NIfTI...
Renombrado NIfTI: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t257.nii.gz -> nifti/0644/4CH/imagenes/time_01_0000.nii.gz
Renombrado y movido JSON: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t257.json -> nifti/0644/4CH/json/time_01_0000.json
Renombrado NIfTI: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t578.nii.gz -> nifti/0644/4CH/imagenes/time_02_0000.nii.gz
Renombrado y movido JSON: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t578.json -> nifti/0644/4CH/json/time_02_0000.json
Renombrado NIfTI: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t96.nii.gz -> nifti/0644/4CH/imagenes/time_03_0000.nii.gz
Renombrado y movido JSON: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t96.json -> nifti/0644/4CH/json/time_03_0000.json
Renombrado NIfTI: nifti/0644/4CH/imagenes/4CH_sBTFE_BH_EC_20230130183159_701_t836.nii.gz -> nifti/0644/4CH/imagenes/time_0

In [15]:
output_zip =  '/content/{}.zip'.format(fl)
output = '/content/nifti/{}'.format(fl)
!zip -r {output_zip} {output}
files.download(output_zip)

  adding: content/nifti/0644/ (stored 0%)
  adding: content/nifti/0644/4CH/ (stored 0%)
  adding: content/nifti/0644/4CH/json/ (stored 0%)
  adding: content/nifti/0644/4CH/json/time_18_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_06_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_01_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_24_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_04_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_12_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_15_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_25_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_11_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_03_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_17_0000.json (deflated 51%)
  adding: content/nifti/0644/4CH/json/time_23_0000.json (deflated 51%)
  adding:

In [17]:
files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files

# Segmentación

In [7]:
# Create directories for nnU-Net
!mkdir -p /content/nnUNet_trained_models
!mkdir -p /content/nnUNet_raw_data_base/nnUNet_raw_data
!mkdir -p /content/nnUNet_preprocessed

# Download the pretrained model (Task027_ACDC) from Zenodo
!wget -O /content/nnUNet_trained_models/Task027_ACDC.zip https://zenodo.org/record/4003545/files/Task027_ACDC.zip
# Unzip the downloaded model files
!unzip /content/nnUNet_trained_models/Task027_ACDC.zip -d /content/nnUNet_trained_models/

--2025-01-08 17:55:19--  https://zenodo.org/record/4003545/files/Task027_ACDC.zip
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4003545/files/Task027_ACDC.zip [following]
--2025-01-08 17:55:20--  https://zenodo.org/records/4003545/files/Task027_ACDC.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1830026265 (1.7G) [application/octet-stream]
Saving to: ‘/content/nnUNet_trained_models/Task027_ACDC.zip’

/content/nnUNet_tra 100%[===================>]   1.70G  12.0MB/s    in 2m 36s  

2025-01-08 17:57:55 (11.2 MB/s) - ‘/content/nnUNet_trained_models/Task027_ACDC.zip’ saved [1830026265/1830026265]

Archive:  /content/nnUNet_trained_models/Task027_ACDC.zip
  inflating: /content/nnUNet_trained_models/ensembles/Task027_ACDC/summary.csv  
  inflati

In [8]:
import os
# Set paths to nnU-Net directories
os.environ['nnUNet_raw_data_base'] = '/content/nnUNet_raw_data_base'
os.environ['nnUNet_preprocessed'] = '/content/nnUNet_preprocessed'
os.environ['RESULTS_FOLDER'] = '/content/nnUNet_trained_models'

In [11]:
!nnUNet_predict -i /content/input_segm -o /content/ouput_segm -t 27 -m 3d_fullres



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/nnUNet_trained_models/nnUNet/3d_fullres/Task027_ACDC/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 6 unique case ids, here are some examples: ['0771_EC_ED' '0768_EC_ED' '0807_EC_ED' '0807_EC_ED' '0787_EC_ED'
 '0787_EC_ED']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 6
number of cases that still need to be predicted: 6
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:

In [12]:
!zip -r /content/segmentacion.zip /content/ouput_segm

  adding: content/ouput_segm/ (stored 0%)
  adding: content/ouput_segm/0807_EC_ED.nii.gz (deflated 22%)
  adding: content/ouput_segm/0787_EC_ED.nii.gz (deflated 24%)
  adding: content/ouput_segm/plans.pkl (deflated 75%)
  adding: content/ouput_segm/0644_EC_ED.nii.gz (deflated 23%)
  adding: content/ouput_segm/0786_EC_ED.nii.gz (deflated 20%)
  adding: content/ouput_segm/prediction_time.txt (stored 0%)
  adding: content/ouput_segm/0768_EC_ED.nii.gz (deflated 26%)
  adding: content/ouput_segm/postprocessing.json (deflated 57%)
  adding: content/ouput_segm/0771_EC_ED.nii.gz (deflated 21%)
